In [4]:
!pip install -qU langchain langchain_chroma langchain_core langchain_community

In [16]:
!pip install -qU langchain-google-genai

In [3]:
import re

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langchain.agents import create_agent

from dotenv import load_dotenv
load_dotenv()


True

# Database Setup

In [4]:
import sqlite3
db = SQLDatabase.from_uri("sqlite:///employees_db-full-1.0.6.db")

try:
    tables = db.get_usable_table_names()
    print("Tables in the database:", tables)
    print("Number of tables:", len(tables))
except Exception as e:
    print("Error retrieving table names:", e)


SCHEMA = db.get_table_info()
print("Database Schema:\n", SCHEMA)

Tables in the database: ['departments', 'dept_emp', 'dept_manager', 'employees', 'salaries', 'titles']
Number of tables: 6
Database Schema:
 
CREATE TABLE departments (
	dept_no CHAR(4) NOT NULL, 
	dept_name VARCHAR(40) NOT NULL, 
	PRIMARY KEY (dept_no), 
	UNIQUE (dept_name)
)

/*
3 rows from departments table:
dept_no	dept_name
d009	Customer Service
d005	Development
d002	Finance
*/


CREATE TABLE dept_emp (
	emp_no INTEGER NOT NULL, 
	dept_no CHAR(4) NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY KEY (emp_no, dept_no), 
	FOREIGN KEY(dept_no) REFERENCES departments (dept_no), 
	FOREIGN KEY(emp_no) REFERENCES employees (emp_no)
)

/*
3 rows from dept_emp table:
emp_no	dept_no	from_date	to_date
10001	d005	1986-06-26	9999-01-01
10002	d007	1996-08-03	9999-01-01
10003	d004	1995-12-03	9999-01-01
*/


CREATE TABLE dept_manager (
	dept_no CHAR(4) NOT NULL, 
	emp_no INTEGER NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY KEY (emp_no, dept_no), 
	

# LLM Initialization

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key="")

response = llm.invoke("Hello")
response.pretty_print()
print("Response Text:", response.text)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=212d3592-93ca-4c49-bc42-af527f835cc2,id=212d3592-93ca-4c49-bc42-af527f835cc2


================================== Ai Message ==================================

Hello! How can I help you today?
Response Text: Hello! How can I help you today?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=212d3592-93ca-4c49-bc42-af527f835cc2,id=212d3592-93ca-4c49-bc42-af527f835cc2


# SQL tools

In [6]:
@tool
def get_database_schema(table_name: str = None) -> str:
    """Get the database schema or schema for a specific table."""
    if table_name:
        try:
            tables = db.get_usable_table_names()
            if table_name.lower() in [t.lower() for t in tables]:
                result = db.get_table_info([table_name])
                print("Retrieved schema for table", table_name)
                return result
            else:
                return f"Error: Table '{table_name}' not found. Available tables: {', '.join(tables)}"
        except Exception as e:
            return e
    else:
        print("Retrieved full database schema")
        return SCHEMA

In [7]:
@tool
def generate_sql_query(question: str, schema_info: None) -> str:
    """Generate a SQL SELECT query from a natrual language question using database schema.
    Always use this after getting schema information."""
    print("Generating SQL query for question:", question)

    schema_to_use = schema_info if schema_info else SCHEMA

    prompt = f"""
                Based on this database schema:
                {schema_to_use}

                Generate a SQL query to answer this question: {question}

                Rules:
                - Use only exisiting columns and tables
                - Add  appropriate WHERE, GROUP BY, ORDER BY clauses as needed
                - Limit results to 10 rows unless specified otherwise
                - Use proper SQL syntax for SQLite

                Return only the SQL query, nothing else.
                """
    try:
        response = llm.invoke(prompt)
        query = response.content.strip()
        print("Generted SQL query")
        return query
    except Exception as e:
        return "Error generating query: {e}"

In [8]:
@tool
def validate_sql_query(query: str) -> str:
    """Validate SQL query for safety and syntax before execution.
    Returns 'Valid: <query>' if safe or 'Error: <message>' if unsafe."""
    print(f"Validating SQL: {query[:50]}...")

    # safety checks
    query = query.strip().lower()

    # Remove common SQL formatting
    clean_query = re.sub(r'```sql\s*', '', query, flags=re.IGNORECASE)
    clean_query = re.sub(r'```\s+', '', clean_query)
    clean_query = clean_query.strip()

    # Block multiple statements
    if clean_query.count(":") > 1 or clean_query.endswith(";") and ";" in clean_query[:-1]:
        return "Error: Multiple SQL statements detected. Only single SELECT queries are allowed."
    
    
    clean_query = clean_query.rstrip(";").strip() # Remove trailing semicolon for parsing

    if not clean_query.startswith("select"):
        return "Error: Only SELECT queries are allowed."
    
    # Block dangerous operations
    dangerous_patterns = [
        r'\b(INSERT|UPDATE|DELETE|DROP|ALTER|TRUNCATE|CREATE|EXECUTE|GRANT|REVOKE)\b',
        r'\b(EXEC|EXECUTE)\b',
        r'--', # SQL comments
        r'/\*', # SQL comments
    ]

    for pattern in dangerous_patterns:
        if re.search(pattern, clean_query, re.IGNORECASE):
            return f"Error: Dangerous operation detected in query: '{pattern}'"
    
    # Basic syntax check (very rudimentary)
    try: 
        # Simple parsing check
        if clean_query.count("(") != clean_query.count(")"):
            return "Error: Mismatched parentheses detected."
        
        if clean_query.count('select') != 1:
            return "Error: Only single SELECT queries are allowed."
    except Exception as e:
        return f"Error: Syntax error detected - {e}"




In [9]:
@tool 
def execute_sql_query(query: str) -> str:
    """Execute a validated SQL query and return results or error."""
    print(f"Executing SQL: {query[:50]}...")

    try:
        # Clean the query
        clean_query = query.strip()
        if clean_query.startswith("Valid: "):
            clean_query = clean_query[7:] # Remove "valid: " prefix
        
        clean_query = re.sub(r'```sql\s*', '', clean_query, flags=re.IGNORECASE)
        clean_query = re.sub(r'```\s+', '', clean_query)
        clean_query = clean_query.rstrip(";").strip() # Remove trailing semicolon for sqlite3

        result = db.run(clean_query)
        print("Query executed successfully")

        if result:
            return f"Query Results:\n{result}"
        else:
            return "Query executed successfully but returned no results."
    except Exception as e:
        error_msg = f"Execution Error: {str(e)}"
        print(f" {error_msg}")
        return error_msg
    

In [10]:
@tool
def fix_sql_error(original_query: str, error_message: str, question: str) -> str:
    """Fix SQL query based on error message using database schema."""
    print("Fixing SQL query based on error:", error_message)

    fix_prompt = f"""
                The following SQL query failed:
                Query: {original_query}
                Error: {error_message}
                Original Question: {question}

                Database Schema:
                {SCHEMA}

                Analyze the error and provide a corrected SQL query that:
                1. Fixes the specific error mentioned
                2. Still answers the original question
                3. Uses only valid table and column names from the schema
                4. Follows SQLite syntax rules

                Returns only the corrected SQL query, nothing else.
                """
    try:
        response = llm.invoke(fix_prompt)
        fixed_query = response.content.strip()
        print("Fixed SQL query")
        return fixed_query
    except Exception as e:
        return f"Error fixing query: {e}"

In [13]:
@tool
def analyze_query_results(question: str, query: str, results: str) -> str:
    """Convert SQL query results into a natural language answer.
    Use this as the final step to provide a user-friendly response."""
    print(" Analyzing results and generating answer for question:", question)

    analyze_prompt = f"""
    Original Question: {question}
    SQL Query Used: {query}
    Query Results: {results}

    Provide a clear, natural language answer to the original question on the query result.
    Be specific and include relevant numbers/data from the results.
    If the results are empty or unclear, mention that as well.
    """

    try:
        response = llm.invoke(analyze_prompt)
        answer = response.content.strip()
        print("Generated final answer")
        return answer
    except Exception as e:
        return f"Error generating answer: {e}"
    
print("Created all SQL workflow tools")

Created all SQL workflow tools


In [ ]:
SQL_SYSTEM_PROMPT = f"""You are an expert SQL agent designed to interact with a SQL database.

database schema:
{SCHEMA}

Your workflow for answering questions:
1. Use `get_database_schema` to retrieve the database schema if needed.
2. Use `generate_sql_query` to create a SQL SELECT query based on the user's question
3. Use `validate_sql_query` to ensure the generated SQL query is safe and syntactically correct.
4. If the query is valid, use `execute_sql_query` to run the query against the database.
5. If execution returns an error, use `fix_sql_error` to correct the query based
6. use `analyze_query_results` to convert the query results into a natural language answer.

Rules:
- Always ensure SQL queries are safe and only SELECT statements.
- Always refer to the database schema to ensure correct table and column names.
- Always follow the workflow steps in order.
- Handle errors gracefully and provide informative messages.
- If you fail after 3 attemps, explain what went wrong and suggest possible fixes.

Available tools:
- get_database_schema(table_name: str = None) -> str
- generate_sql_query(question: str, schema_info: None) -> str
- validate_sql_query(query: str) -> str
- execute_sql_query(query: str) -> str
- fix_sql_error(original_query: str, error_message: str, question: str) -> str
- analyze_query_results(question: str, query: str, results: str) -> str

Remember: Always validate queries before execution and fix any errors that arise.
"""

In [15]:
tools = [
    get_database_schema,
    generate_sql_query,
    validate_sql_query,
    fix_sql_error,
    execute_sql_query,
    analyze_query_results
]

In [51]:
from langchain.agents import create_agent
sql_agent = create_agent(
    llm,
    tools,
    system_prompt=SQL_SYSTEM_PROMPT.format(SCHEMA=SCHEMA)
)


# Test Agent

In [52]:
sql_agent

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [61]:
def ask_sql(question: str) -> str:
    """Function to ask a question to the SQL agent and get an answer."""
    for event in sql_agent.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values"
    ): 
        msg = event["messages"][-1]

        # Show tool usage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            tool_call = msg.tool_calls[-1]
            tool_name = tool_call['name']
            tool_args = str(tool_call['args'])
            if len(tool_args) > 200:
                tool_args = tool_args[:200] + "..."
            print(f"\n[Tool Used: {tool_name}]\nArgs: {tool_args}\n")
        elif hasattr(msg, 'content') and msg.content:
            print(f"\n[Agent Answer]: {msg.content}\n")


In [62]:
ask_sql("List the top 5 highest paid employees in each department.")


[Agent Answer]: List the top 5 highest paid employees in each department.



Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=8198e06d-dbfb-4eac-a5bb-38899cdcfb89; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=4f6303e8-d9d5-401c-9587-ef878fcee6d9; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=7e4a9168-8a9a-4e22-812c-b437be5f9c5b



[Tool Used: get_database_schema]
Args: {}

Retrieved full database schema

[Agent Answer]: 
CREATE TABLE departments (
	dept_no CHAR(4) NOT NULL, 
	dept_name VARCHAR(40) NOT NULL, 
	PRIMARY KEY (dept_no), 
	UNIQUE (dept_name)
)

/*
3 rows from departments table:
dept_no	dept_name
d009	Customer Service
d005	Development
d002	Finance
*/


CREATE TABLE dept_emp (
	emp_no INTEGER NOT NULL, 
	dept_no CHAR(4) NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY KEY (emp_no, dept_no), 
	FOREIGN KEY(dept_no) REFERENCES departments (dept_no), 
	FOREIGN KEY(emp_no) REFERENCES employees (emp_no)
)

/*
3 rows from dept_emp table:
emp_no	dept_no	from_date	to_date
10001	d005	1986-06-26	9999-01-01
10002	d007	1996-08-03	9999-01-01
10003	d004	1995-12-03	9999-01-01
*/


CREATE TABLE dept_manager (
	dept_no CHAR(4) NOT NULL, 
	emp_no INTEGER NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY KEY (emp_no, dept_no), 
	FOREIGN KEY(dept_no) REFERENCES departments (dept

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=7e4a9168-8a9a-4e22-812c-b437be5f9c5b; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=21932997-5168-4c37-aec2-80b668c8c995; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=21932997-5168-4c37-aec2-80b668c8c995; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=4f6303e8-d9d5-401c-9587-ef878fcee6d9; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=272f237c-064e-4d8e-83f0-b5dbb9e38203; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=3766fca0-9139-4bba-956c-ae203b06fb4e; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=3766fca0-9139-4bba-956c-ae203b06fb4e; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=98d21550-5596-4516-ab01-1ef072aed7aa; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id


[Tool Used: generate_sql_query]
Args: {'question': 'List the top 5 highest paid employees in each department.', 'schema_info': '\nCREATE TABLE departments (\n\tdept_no CHAR(4) NOT NULL, \n\tdept_name VARCHAR(40) NOT NULL, \n\tPRIMARY KEY ...


[Agent Answer]: Error invoking tool 'generate_sql_query' with kwargs {'question': 'List the top 5 highest paid employees in each department.', 'schema_info': '\nCREATE TABLE departments (\n\tdept_no CHAR(4) NOT NULL, \n\tdept_name VARCHAR(40) NOT NULL, \n\tPRIMARY KEY (dept_no), \n\tUNIQUE (dept_name)\n)\n\n/*\n3 rows from departments table:\ndept_no\tdept_name\nd009\tCustomer Service\nd005\tDevelopment\nd002\tFinance\n*/\n\n\nCREATE TABLE dept_emp (\n\temp_no INTEGER NOT NULL, \n\tdept_no CHAR(4) NOT NULL, \n\tfrom_date DATE NOT NULL, \n\tto_date DATE NOT NULL, \n\tPRIMARY KEY (emp_no, dept_no), \n\tFOREIGN KEY(dept_no) REFERENCES departments (dept_no), \n\tFOREIGN KEY(emp_no) REFERENCES employees (emp_no)\n)\n\n/*\n3 rows from dept_emp table:\

...

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=4de1caba-659d-4aa9-b425-1373df1eb88a; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=15014602-074b-4a45-bc72-a24747d315ab; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=15014602-074b-4a45-bc72-a24747d315ab; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=976bb48b-bf60-4243-bc3f-1bf69193ed6a; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=fd189203-7108-4214-a7b9-489e4956a02f; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=8236cc24-5443-4e11-a114-130021f7408b; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=fd91a02c-58a3-4615-92c8-02e4ced82b2a


Generated final answer

[Agent Answer]: Here are the top 5 highest-paid employees in each department, based on their current salaries:

**Customer Service:**
1.  Vidya Hanabata: 144,866
2.  Odoardo Lindenbaum: 143,950
3.  Seongbin Mitsuhashi: 143,937
4.  Bedrich Luft: 143,832
5.  Yefim Taneja: 141,555

**Development:**
1.  Khosrow Sgarro: 144,434
2.  Jixiang Rindone: 140,784
3.  Dayanand Morrey: 136,130
4.  Reuven Uludag: 134,570
5.  Moon Uhrik: 133,712

**Finance:**
1.  Lunjin Swick: 142,395
2.  Uta Lowrie: 140,742
3.  Genki Delaune: 138,775
4.  Waiman Lodder: 136,978
5.  Tetsushi Staylopatis: 135,857

**Human Resources:**
1.  Yinlin Flowers: 141,953
2.  Kannan Emiris: 128,308
3.  Yifei Gien: 125,263
4.  Shan Encarnacion: 123,674
5.  Yinghua Maraist: 123,268

**Marketing:**
1.  Akemi Warwick: 145,128
2.  Ingmar Isard: 143,644
3.  Ortrud Diderrich: 142,506
4.  Chikako Jiang: 142,485
5.  Clyde Georgakopoulos: 141,842

**Production:**
1.  Youjian Cronau: 138,273
2.  Shimshon Azuma: 137,5

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=fd91a02c-58a3-4615-92c8-02e4ced82b2a; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=8236cc24-5443-4e11-a114-130021f7408b; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=4660e140-ac3c-4d3c-8e78-8b94b5b0c1b2; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=4660e140-ac3c-4d3c-8e78-8b94b5b0c1b2; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=fd189203-7108-4214-a7b9-489e4956a02f; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=d27aa63a-491d-4957-8326-d329413193bb; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=e760a655-84d0-4ed0-aefc-3590832efac2



[Agent Answer]: [{'type': 'text', 'text': 'Here are the top 5 highest-paid employees in each department, based on their current salaries:\n\n**Customer Service:**\n1. Vidya Hanabata: 144,866\n2. Odoardo Lindenbaum: 143,950\n3. Seongbin Mitsuhashi: 143,937\n4. Bedrich Luft: 143,832\n5. Yefim Taneja: 141,555\n\n**Development:**\n1. Khosrow Sgarro: 144,434\n2. Jixiang Rindone: 140,784\n3. Dayanand Morrey: 136,130\n4. Reuven Uludag: 134,570\n5. Moon Uhrik: 133,712\n\n**Finance:**\n1. Lunjin Swick: 142,395\n2. Uta Lowrie: 140,742\n3. Genki Delaune: 138,775\n4. Waiman Lodder: 136,978\n5. Tetsushi Staylopatis: 135,857\n\n**Human Resources:**\n1. Yinlin Flowers: 141,953\n2. Kannan Emiris: 128,308\n3. Yifei Gien: 125,263\n4. Shan Encarnacion: 123,674\n5. Yinghua Maraist: 123,268\n\n**Marketing:**\n1. Akemi Warwick: 145,128\n2. Ingmar Isard: 143,644\n3. Ortrud Diderrich: 142,506\n4. Chikako Jiang: 142,485\n5. Clyde Georgakopoulos: 141,842\n\n**Production:**\n1. Youjian Cronau: 138,273\n2. Shims

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=e760a655-84d0-4ed0-aefc-3590832efac2; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=b72b5f47-c35b-44eb-81e9-0c80dbe50129; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=b72b5f47-c35b-44eb-81e9-0c80dbe50129; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=d27aa63a-491d-4957-8326-d329413193bb; trace=8198e06d-dbfb-4eac-a5bb-38899cdcfb89,id=8198e06d-dbfb-4eac-a5bb-38899cdcfb89
